In [1]:
import os
from numba import jit
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import scipy
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score,f1_score, roc_curve, auc

In [2]:
rcParams['font.family']='Times New Roman'

In [3]:
entropy_grey = np.load('./results/entropy_grey.npy', allow_pickle=True)
entropy2d_grey = np.load('./results/entropy2d_grey.npy', allow_pickle=True)
entropy_hue = np.load('./results/entropy_hue.npy', allow_pickle=True)
tone_HSVs = np.load('./results/tone_HSVs.npy', allow_pickle=True)    # special handle
color_clusters = np.load('./results/color_clusters.npy', allow_pickle=True)
areas = np.load('./results/areas.npy', allow_pickle=True)
horizon_intervals = np.load('./results/horizon_intervals.npy', allow_pickle=True)
vertical_intervals = np.load('./results/vertival_intervals.npy', allow_pickle=True)

for feature in [entropy_grey, entropy2d_grey, entropy_hue, tone_HSVs, color_clusters, areas, horizon_intervals, vertical_intervals]:
    feature[0] = [v for i, v in enumerate(feature[0]) if i+1 not in {177,205,257,302}]
    feature[1] = [v for i, v in enumerate(feature[1]) if i+1 not in {78,79}]

features = [entropy_grey, entropy2d_grey, entropy_hue, color_clusters, areas, horizon_intervals, vertical_intervals]

In [4]:
def HSV2cone(points):
    points = points.copy().astype(np.double)
    points[:,0] = 2 * np.deg2rad(points[:,0])
    points[:,1] /=255; points[:,2] /= 255    # uniformization
    
    new_points = np.zeros(points.shape, dtype=np.double)
    points_sv = points[:, 1] * points[:, 2]
    
    new_points[:,0] = points_sv * np.cos(points[:, 0])
    new_points[:,1] = points_sv * np.sin(points[:, 0])
    new_points[:,2] = points[:, 2]
    return new_points

In [5]:
ASD_HSV = np.array(tone_HSVs[0])
TD_HSV = np.array(tone_HSVs[1])
ASD_XYZ = HSV2cone(ASD_HSV)
TD_XYZ = HSV2cone(TD_HSV)

In [6]:
tbs = [np.zeros(len(entropy_grey[0])).reshape(-1,1), np.zeros(len(entropy_grey[1])).reshape(-1,1)]

for j,feature in enumerate(features):
    for i in range(2):
        tbs[i] = np.hstack((tbs[i], np.array(feature[i]).reshape(-1,1)))
tbs = np.vstack((tbs[0][:,1:], tbs[1][:,1:]))

In [7]:
tb = pd.DataFrame(tbs)

tb['X'] = np.hstack((ASD_XYZ[:,0],TD_XYZ[:,0])).reshape(-1,1)
tb['Y'] = np.hstack((ASD_XYZ[:,1],TD_XYZ[:,1])).reshape(-1,1)
tb['Z'] = np.hstack((ASD_XYZ[:,2],TD_XYZ[:,2])).reshape(-1,1)
tb['label'] = np.vstack((np.ones(len(entropy_grey[0]), dtype=np.uint8).reshape(-1,1), np.zeros(len(entropy_grey[1]), dtype=np.uint8).reshape(-1,1)))
tb.columns = ['1D Gray Entropy', '2D Gray Entropy', '1D Hue Entropy', 'Color Richness', 'Area', 'Horizontal Intervals', 'Vertical Intervals', 'Dominant X', 'Dominant Y', 'Dominant Z', 'label']
tb.to_csv('./results/all_features.csv')

In [8]:
tb

,1D Gray Entropy,2D Gray Entropy,1D Hue Entropy,Color Richness,Area,Horizontal Intervals,Vertical Intervals,Dominant X,Dominant Y,Dominant Z,label
0,3.757319,8.199638,2.984306,4.0,0.180172,0.000000,0.009840,-0.022779,-0.031353,0.823529,1
1,4.714888,9.984443,4.135668,3.0,0.983521,0.003626,0.006752,-0.045593,-0.102404,0.772549,1
2,5.070659,10.304571,4.157235,4.0,0.995805,0.003348,0.007573,-0.014302,0.204529,0.396078,1
3,4.861554,9.968199,4.632747,4.0,0.995803,0.002196,0.003786,-0.034903,-0.078394,0.235294,1
4,4.831612,9.891949,4.300436,2.0,0.976716,0.002347,0.006901,-0.122485,0.230361,0.458824,1
...,...,...,...,...,...,...,...,...,...,...,...
530,4.472156,9.025126,3.321349,7.0,0.891165,0.000057,0.000552,-0.015255,-0.046950,0.839216,0
531,4.658013,8.932943,3.891208,3.0,0.931654,0.000646,0.001522,-0.009792,-0.034149,0.823529,0
532,4.726424,8.967950,4.560189,7.0,0.973873,0.005552,0.008712,0.058039,0.000000,0.870588,0
533,4.792121,9.414940,4.616940,5.0,0.973879,0.001028,0.003203,-0.032092,0.028895,0.211765,0


In [15]:
accuracy = []
precision = []
recall = []
F1 = []

kf = KFold(n_splits=5, shuffle=True, random_state=19260817)
for train_index, test_index in kf.split(tb):
    train_X = tb.iloc[:,:-1].iloc[train_index]
    test_X = tb.iloc[:,:-1].iloc[test_index]
    train_y = tb.iloc[:,-1][train_index]
    test_y = tb.iloc[:,-1][test_index]

    svm_clf = svm.SVC(kernel='rbf', gamma=0.6)
    svm_clf.fit(train_X,train_y)
    test_pred = svm_clf.predict(test_X)
    
    accuracy.append(accuracy_score(test_y, test_pred))
    precision.append(precision_score(test_y, test_pred))
    recall.append(recall_score(test_y, test_pred))
    F1.append(f1_score(test_y, test_pred))
print(np.mean(accuracy), np.mean(precision), np.mean(recall), np.mean(F1))
print(np.std(accuracy), np.std(precision), np.std(recall), np.std(F1))

0.8598130841121495 0.864000465990934 0.9492866771347783 0.9045521932880399
0.01182159872960142 0.014011416593053978 0.0103586902862562 0.009095546653007925
